In [187]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
from pandas import read_csv
import logging
from functools import reduce
import csv
from collections import defaultdict
import os
import uuid

In [219]:
etl = 'cuartero2newaqm'

input_schema_file = ''
input_data_file = ''
mapping_file = ''

datelog_dir = 'couchbase-curis-2019-06-21-cuartero'

if etl == 'cuartero2newaqm':
    schema_meta_file = '_meta.csv'
    schema_meta_dir = 'data/processed/' + datelog_dir + '/'
    schema_meta_path = schema_meta_dir + schema_meta_file
    
    #mapping_file = '2.1.TestAQMHealthInfoQuestions.V1.map.csv'
    #mapping_file = '2.1.AQMGeneralQuestions.V1.map.csv'
    mapping_file = '2.2.AQMHealthInfoQuestions.V1.map.csv'
    #mapping_file = '2.6.AQMMentallHealthQuestions.V1.map.csv'
    #mapping_file = '2.7.AWHDisabilityQuestions.V1.map.csv'
    #mapping_file = '1.1.AQMPersonalQuestions.V1.map.csv'
    
    mapping_dir = 'schema/map/Philippines/' 
    mapping_path = mapping_dir + mapping_file
    
    #TODO: add cleaned_dir
    tmp_dir = 'data/processed/' + datelog_dir + '/tmp/'
    processed_dir = 'data/cleaned/' + datelog_dir + '/'
    merged_dir = 'data/merged/' + datelog_dir + '/'
    
    

## TODO: datecreated date format must be UTF8

## TODO: DISABILTY CAUSE MUST BE ARRAY NOT STRING!!

## TODO: CREATE FOLDER DIRECTORY MKDIR

## TODO: FIX ID MERGING

## TODO: FIX _id into profileId

# READ MAPPING FILE

In [189]:
#def _get_mapping_fields():
_mapping_df = pd.DataFrame()
_mapping_df = read_csv(mapping_path).sort_values(['source_key']).replace(np.nan,'',regex=True)
_mapping_df.head(3)

,source_key,source_type,destination_key,destination_type,data_type,data_source,data_format,default_value,lookup_value
9,gender,string,gender,string,existing,original,,,
0,health_informations.allergies,string,answers.allergies,string,existing,original,,,
2,health_informations.blood_pressure.first_readi...,string,answers.bp1Diastole,string,existing,original,,,


# GET THE FIELDS IN THE MAPPING FILE

In [190]:
mapping_fields_list = []
mapping_fields_list = list(filter(None, (_mapping_df['source_key'].unique())))
mapping_fields_list

['gender',
 'health_informations.allergies',
 'health_informations.blood_pressure.first_reading.diastole',
 'health_informations.blood_pressure.first_reading.systole',
 'health_informations.blood_sign',
 'health_informations.blood_sugar',
 'health_informations.exercise_in_a_week',
 'health_informations.family_history',
 'health_informations.smoking_habit',
 'profiles.civil_status',
 'registered_at',
 'user-cam.id']

# GET THE FIELDS IN THE SCHEMA META (i.e. _meta) FILE

In [191]:
meta_headers_df = pd.DataFrame()
meta_headers_df = pd.read_csv(schema_meta_path)
meta_headers_df.head(5)

,file_name,field_name,field_type
0,resident,middle_name,primitive
1,resident,birthdate,primitive
2,resident,id,primitive
3,resident,type,primitive
4,resident,first_name,primitive


## MATCH THE FIELDS IN MAPPING FILE AND SCHEMA META FILE

In [192]:
match_headers_df = pd.DataFrame()
match_headers_df = meta_headers_df[meta_headers_df['field_name'].isin(mapping_fields_list)]
match_headers_df = match_headers_df.sort_values(['file_name','field_name']).reset_index(drop=True)
match_headers_df

,file_name,field_name,field_type
0,health_informations,health_informations.allergies,primitive
1,health_informations,health_informations.blood_pressure.first_readi...,primitive
2,health_informations,health_informations.blood_pressure.first_readi...,primitive
3,health_informations,health_informations.blood_sign,primitive
4,health_informations,health_informations.exercise_in_a_week,primitive
5,health_informations,health_informations.smoking_habit,primitive
6,health_informations.family_history,health_informations.family_history,list
7,profiles,profiles.civil_status,primitive
8,resident,gender,primitive
9,resident,registered_at,primitive


# CREATE DEFAULT DICT FOR FILENAME AS KEY and FIELD NAMES AS VALUE

In [193]:
filename_per_field_dd = defaultdict(list)

for index,row in match_headers_df.iterrows():
    filename = row['file_name']
    fields = row['field_name']
    filename_per_field_dd[filename].append(fields)  
    
filename_per_field_dd

defaultdict(list,
            {'health_informations': ['health_informations.allergies',
              'health_informations.blood_pressure.first_reading.diastole',
              'health_informations.blood_pressure.first_reading.systole',
              'health_informations.blood_sign',
              'health_informations.exercise_in_a_week',
              'health_informations.smoking_habit'],
             'health_informations.family_history': ['health_informations.family_history'],
             'profiles': ['profiles.civil_status'],
             'resident': ['gender', 'registered_at', 'user-cam.id']})

## CREATE OUTPUT FILE WITH DYNAMIC NAME DERIVED FROM MAPPING FILE

In [194]:
_output_filename = mapping_file.split(sep='.')[2]
_output_filename

'TestAQMHealthInfoQuestions'

## REMOVE OUTPUT FILE IF EXISTING

In [195]:
if os.path.exists(merged_dir + _output_filename + '.csv'):
    os.remove(merged_dir + _output_filename + '.csv' )

## WRITE EMPTY CSV FOR MERGE.csv

### TODO: FILENAME MUST BE DYNAMIC

## HARDCODE TEST DATA

In [196]:
test_filename_per_field_dd = {
    'health_informations': ['health_informations.allergies',
              'health_informations.blood_pressure.first_reading.diastole',
              'health_informations.blood_pressure.first_reading.systole',
              'health_informations.blood_pressure.second_reading.diastole',
              'health_informations.blood_pressure.second_reading.systole',
              'health_informations.blood_sign',
              'health_informations.blood_type',
              'health_informations.exercise_in_a_week',
              'health_informations.smoking_habit'],
    'health_informations.family_history': ['health_informations.family_history']}

test_filename_per_field_dd = {'profiles': ['profiles.civil_status','profiles.employment.is_employed','profiles.education','profiles.employment.nature','profiles.religion']}
test_filename_per_field_dd = {
        'health_informations': [
                  'health_informations.allergies',
                  'health_informations.blood_pressure.first_reading.diastole',
                  'health_informations.blood_pressure.first_reading.systole',
                  'health_informations.blood_sign',
                  'health_informations.exercise_in_a_week',
                  'health_informations.smoking_habit'],
        'health_informations.family_history': [
                  'health_informations.family_history'],
        'profiles': ['profiles.civil_status'],
        'resident': ['gender','registered_at','user-cam.id']}

## DYNAMIC TEST DATA

In [197]:
test_filename_per_field_dd = filename_per_field_dd

## CREATE OUTPUT FILE WITH HEADERS BASED ON _META AND MAPPING FILE

In [198]:
fields_list = list(test_filename_per_field_dd.values())
flat_fields_list = [item for sublist in fields_list for item in sublist]
all_fields_list  = ['_id','_index_map'] + flat_fields_list

empty_data_df = pd.DataFrame(columns=all_fields_list)
empty_data_df.to_csv(merged_dir + _output_filename + '.csv', encoding='utf-8', mode='w', header=True,index=False)
empty_data_df

,_id,_index_map,health_informations.allergies,health_informations.blood_pressure.first_reading.diastole,health_informations.blood_pressure.first_reading.systole,health_informations.blood_sign,health_informations.exercise_in_a_week,health_informations.smoking_habit,health_informations.family_history,profiles.civil_status,gender,registered_at,user-cam.id


## TODO: DYNAMICALLY MERGE DATA FROM DIFFERENT 

In [199]:
fields_list = list(test_filename_per_field_dd.values())
flat_fields_list = [item for sublist in fields_list for item in sublist]
flat_fields_list

['health_informations.allergies',
 'health_informations.blood_pressure.first_reading.diastole',
 'health_informations.blood_pressure.first_reading.systole',
 'health_informations.blood_sign',
 'health_informations.exercise_in_a_week',
 'health_informations.smoking_habit',
 'health_informations.family_history',
 'profiles.civil_status',
 'gender',
 'registered_at',
 'user-cam.id']

In [200]:
list(test_filename_per_field_dd.items())

[('health_informations',
  ['health_informations.allergies',
   'health_informations.blood_pressure.first_reading.diastole',
   'health_informations.blood_pressure.first_reading.systole',
   'health_informations.blood_sign',
   'health_informations.exercise_in_a_week',
   'health_informations.smoking_habit']),
 ('health_informations.family_history',
  ['health_informations.family_history']),
 ('profiles', ['profiles.civil_status']),
 ('resident', ['gender', 'registered_at', 'user-cam.id'])]

## TODO: identify here if primitive or list. if list .agg to_list

In [201]:
match_headers_df

,file_name,field_name,field_type
0,health_informations,health_informations.allergies,primitive
1,health_informations,health_informations.blood_pressure.first_readi...,primitive
2,health_informations,health_informations.blood_pressure.first_readi...,primitive
3,health_informations,health_informations.blood_sign,primitive
4,health_informations,health_informations.exercise_in_a_week,primitive
5,health_informations,health_informations.smoking_habit,primitive
6,health_informations.family_history,health_informations.family_history,list
7,profiles,profiles.civil_status,primitive
8,resident,gender,primitive
9,resident,registered_at,primitive


In [202]:
filename
match_headers_df[match_headers_df['file_name'] == filename][['field_type','file_name']]

,field_type,file_name
8,primitive,resident
9,primitive,resident
10,primitive,resident


In [203]:
list(match_headers_df[match_headers_df['file_name'] == filename][['field_type','file_name']]['field_type'].unique())

['primitive']

## TODO: optimize here. tmp_df will run out of memory

## TODO: TODO: if index_map = 0, replace with empty

In [204]:
def _array_to_list(_test_df):
    return _test_df.groupby('_id').agg(lambda x: x.tolist())

In [205]:
def _flatten_index_map(index):
    _new_index = ''
    
    if index == '0':
        _new_index = '' 
    elif isinstance(index,list):
        _new_index = ''
    else:
        _new_index = index
        
    return _new_index

In [206]:
_tmp_df = pd.DataFrame(columns=['_id','_index_map'])
_tmp_df.to_csv(merged_dir + _output_filename + '.csv',index=False)

for filename, fields in test_filename_per_field_dd.items():
    
    _mandatory_fields = ['_id','_index_map']
    _fields = list(fields + _mandatory_fields)
    
    _field_type = list(match_headers_df[match_headers_df['file_name'] == filename][['field_type','file_name']]['field_type'].unique())[0]
    
    
    _test_df = pd.DataFrame()
    _test_df = pd.read_csv(processed_dir + str(filename) + '.csv', dtype={"_index_map": str}).sort_values(['_id','_index_map']) 
    
    _test_df = _test_df[_fields].replace(np.nan,'',regex=True).reset_index(drop=True)
    
    if _field_type == 'list':
        _test_df = _array_to_list(_test_df)

    _test_df['_index_map'] = _test_df['_index_map'].apply(_flatten_index_map)
    
    print('Start Merging ', filename)
    _tmp_df = _tmp_df.merge(_test_df,on=["_id","_index_map"], how="outer",  suffixes=('_x', '_y') ).replace(np.nan,'',regex=True)
    

_tmp_df.to_csv(merged_dir + _output_filename + '.csv',index=False)
_tmp_df.T


Start Merging  health_informations
Start Merging  health_informations.family_history
Start Merging  profiles
Start Merging  resident


,0,1,2,3,4,5,6,7,8,9,...,20284,20285,20286,20287,20288,20289,20290,20291,20292,20293
health_informations.allergies,,,,,,,,,,no,...,,,,,,,,,,
health_informations.blood_pressure.first_reading.diastole,,,,,,,,,80,80,...,,,,,,,,,,
health_informations.blood_pressure.first_reading.systole,,,,,,,,,110,110,...,,,,,,,,,,
health_informations.blood_sign,empty,empty,empty,Wala ko kabalo,Wala ko kabalo,Wala ko kabalo,empty,empty,empty,Wala ko kabalo,...,,,,,,,,,,
health_informations.exercise_in_a_week,0 to 1 hour,0 to 1 hour,0 to 1 hour,,,,0 to 1 hour,0 to 1 hour,0 to 1 hour,0 to 1 Oras,...,,,,,,,,,,
health_informations.smoking_habit,Never,Never,Never,empty,empty,Hindi,Never,Never,Stopped for more than a year,Hindi,...,,,,,,,,,,
_id,0003ff38-28fb-4005-9437-d276cbb9da4d,0003ff38-28fb-4005-9437-d276cbb9da4d,0003ff38-28fb-4005-9437-d276cbb9da4d,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0006de3c-57ea-46d2-89b8-e606533d3d15,0006de3c-57ea-46d2-89b8-e606533d3d15,000c23e6-6f0b-45ab-a0b5-e5d19dc023ad,000cb28d-04d8-4665-90e6-fb1e2fe3c11c,...,fcd4da89-2da9-4449-914e-0af73ccfd03e,fcf1e737-c9f6-4362-9a45-0c1482f408c3,fd5ddbce-c75f-4ecb-8417-78bff115c40b,fdc4dc60-e52b-4f67-afe9-1569ebb727d9,fe833db0-1b43-4259-9276-16ee9ae71a4d,feb16cba-0ff2-4e60-b469-84c9c085ce98,fec1152d-4585-441d-96e2-2bc5ef133bac,fec85354-cac2-442c-b003-221a5cb19fdc,ff4cb0db-36fc-4f98-9bfd-db104aec90d4,ff667f61-3db1-4949-b40c-c29f81a56071
_index_map,,1,2,,1,2,,1,,,...,,,,,,,,,,
health_informations.family_history,"[Kidney disease, Kidney disease, Kidney disease]",,,,,,"[ubo/sipon, Stroke]",,"[Diabetes, Respiratory disease]",,...,,,,,,,,,,
profiles.civil_status,Single,Single,,Single,Single,Single,Single,Single,Married,Widowed,...,,,,,,,,,,


## WRITE MERGE DATA TO CSV

In [207]:
test_group_df = pd.read_csv(merged_dir + _output_filename + '.csv',dtype={'_index_map': str}).replace(np.nan,'',regex=True)

In [208]:
test_group_df.T

,0,1,2,3,4,5,6,7,8,9,...,20284,20285,20286,20287,20288,20289,20290,20291,20292,20293
health_informations.allergies,,,,,,,,,,no,...,,,,,,,,,,
health_informations.blood_pressure.first_reading.diastole,,,,,,,,,80,80,...,,,,,,,,,,
health_informations.blood_pressure.first_reading.systole,,,,,,,,,110,110,...,,,,,,,,,,
health_informations.blood_sign,empty,empty,empty,Wala ko kabalo,Wala ko kabalo,Wala ko kabalo,empty,empty,empty,Wala ko kabalo,...,,,,,,,,,,
health_informations.exercise_in_a_week,0 to 1 hour,0 to 1 hour,0 to 1 hour,,,,0 to 1 hour,0 to 1 hour,0 to 1 hour,0 to 1 Oras,...,,,,,,,,,,
health_informations.smoking_habit,Never,Never,Never,empty,empty,Hindi,Never,Never,Stopped for more than a year,Hindi,...,,,,,,,,,,
_id,0003ff38-28fb-4005-9437-d276cbb9da4d,0003ff38-28fb-4005-9437-d276cbb9da4d,0003ff38-28fb-4005-9437-d276cbb9da4d,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,0006de3c-57ea-46d2-89b8-e606533d3d15,0006de3c-57ea-46d2-89b8-e606533d3d15,000c23e6-6f0b-45ab-a0b5-e5d19dc023ad,000cb28d-04d8-4665-90e6-fb1e2fe3c11c,...,fcd4da89-2da9-4449-914e-0af73ccfd03e,fcf1e737-c9f6-4362-9a45-0c1482f408c3,fd5ddbce-c75f-4ecb-8417-78bff115c40b,fdc4dc60-e52b-4f67-afe9-1569ebb727d9,fe833db0-1b43-4259-9276-16ee9ae71a4d,feb16cba-0ff2-4e60-b469-84c9c085ce98,fec1152d-4585-441d-96e2-2bc5ef133bac,fec85354-cac2-442c-b003-221a5cb19fdc,ff4cb0db-36fc-4f98-9bfd-db104aec90d4,ff667f61-3db1-4949-b40c-c29f81a56071
_index_map,,1,2,,1,2,,1,,,...,,,,,,,,,,
health_informations.family_history,"['Kidney disease', 'Kidney disease', 'Kidney d...",,,,,,"['ubo/sipon', 'Stroke']",,"['Diabetes', 'Respiratory disease']",,...,,,,,,,,,,
profiles.civil_status,Single,Single,,Single,Single,Single,Single,Single,Married,Widowed,...,,,,,,,,,,


## RENAME THE HEADERS USING MAPPED HEADERS

In [209]:
source_destination_keys_df = pd.DataFrame()
source_destination_keys_df = _mapping_df[['source_key','destination_key']]

In [210]:
new_column_name_dict = dict(zip(source_destination_keys_df['source_key'], source_destination_keys_df['destination_key']))
new_column_name_dict 

{'gender': 'gender',
 'health_informations.allergies': 'answers.allergies',
 'health_informations.blood_pressure.first_reading.diastole': 'answers.bp1Diastole',
 'health_informations.blood_pressure.first_reading.systole': 'answers.bp1Systole',
 'health_informations.blood_sign': 'answers.rhesusType',
 'health_informations.blood_sugar': 'answers.bloodSugar',
 'health_informations.exercise_in_a_week': 'answers.weeklyExercise',
 'health_informations.family_history': 'answers.familyDiagnosis',
 'health_informations.smoking_habit': 'answers.smokingHabit',
 'profiles.civil_status': 'answers.civilStatus',
 'registered_at': 'dateCreated',
 'user-cam.id': 'createdBy',
 '': 'type'}

## GET THE DATA from merged dataframes

In [211]:
_required_data_df = pd.DataFrame()
_required_data_df = test_group_df

## APPLICABLE only if not Resident data

## DO NECESSARY TRANSFORMATION

In [212]:
_required_data_df.rename(columns = new_column_name_dict,inplace=True )

## APPLICABLE only if NOT Resident data
_required_data_df.rename(columns={'_id': 'profileId'}, inplace=True)
_required_data_df.head(3)

,answers.allergies,answers.bp1Diastole,answers.bp1Systole,answers.rhesusType,answers.weeklyExercise,answers.smokingHabit,profileId,_index_map,answers.familyDiagnosis,answers.civilStatus,gender,dateCreated,createdBy
0,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,,"['Kidney disease', 'Kidney disease', 'Kidney d...",Single,Male,07/22/2018 at 08:37:43 PM GMT+08:00,kristel@gmail.com
1,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,1,,Single,,,
2,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,2,,,,,


## GENERATE _id column

In [213]:
_required_data_df['_id'] = _required_data_df.index.to_series().map(lambda x: uuid.uuid4())
_required_data_df['id'] = _required_data_df['_id']
_required_data_df

,answers.allergies,answers.bp1Diastole,answers.bp1Systole,answers.rhesusType,answers.weeklyExercise,answers.smokingHabit,profileId,_index_map,answers.familyDiagnosis,answers.civilStatus,gender,dateCreated,createdBy,_id,id
0,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,,"['Kidney disease', 'Kidney disease', 'Kidney d...",Single,Male,07/22/2018 at 08:37:43 PM GMT+08:00,kristel@gmail.com,758c968a-4bad-4ffd-83ed-16b105d12404,758c968a-4bad-4ffd-83ed-16b105d12404
1,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,1,,Single,,,,329aef98-ebb4-4188-8a45-08bd954d714d,329aef98-ebb4-4188-8a45-08bd954d714d
2,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,2,,,,,,ab191553-b31f-41a7-9436-dd1298026f24,ab191553-b31f-41a7-9436-dd1298026f24
3,,,,Wala ko kabalo,,empty,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,,Single,Babae,07/16/2018 at 11:53:05 AM GMT+08:00,geralyn@gmail.com,cb115c52-5303-4f16-a679-dd8ff5df31eb,cb115c52-5303-4f16-a679-dd8ff5df31eb
4,,,,Wala ko kabalo,,empty,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,1,,Single,,,,37fbcb0a-0fab-4c7e-9ac6-9c371baf096d,37fbcb0a-0fab-4c7e-9ac6-9c371baf096d
5,,,,Wala ko kabalo,,Hindi,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,2,,Single,,,,a3757c6e-e13e-4ddd-b7ab-ad2ff1b40a5e,a3757c6e-e13e-4ddd-b7ab-ad2ff1b40a5e
6,,,,empty,0 to 1 hour,Never,0006de3c-57ea-46d2-89b8-e606533d3d15,,"['ubo/sipon', 'Stroke']",Single,Male,07/25/2018 at 10:35:28 AM GMT+08:00,kristel@gmail.com,e26302bb-49d4-4c94-a291-fa28073b9675,e26302bb-49d4-4c94-a291-fa28073b9675
7,,,,empty,0 to 1 hour,Never,0006de3c-57ea-46d2-89b8-e606533d3d15,1,,Single,,,,c2bf04be-e637-4b72-ab5d-696ed8ada1db,c2bf04be-e637-4b72-ab5d-696ed8ada1db
8,,80,110,empty,0 to 1 hour,Stopped for more than a year,000c23e6-6f0b-45ab-a0b5-e5d19dc023ad,,"['Diabetes', 'Respiratory disease']",Married,Male,07/25/2018 at 11:06:17 AM GMT+08:00,silver@gmail.com,2a1c6260-a84c-441a-8d02-e3e5d74fd48b,2a1c6260-a84c-441a-8d02-e3e5d74fd48b
9,no,80,110,Wala ko kabalo,0 to 1 Oras,Hindi,000cb28d-04d8-4665-90e6-fb1e2fe3c11c,,,Widowed,Babae,07/14/2018 at 11:45:05 AM GMT+08:00,bonbon@gmail.com,6ce8af76-f6f0-4021-9662-6e75ce696d00,6ce8af76-f6f0-4021-9662-6e75ce696d00


## CLEAR _INDEX_MAP since each _id has generaed ID

In [214]:
_required_data_df['_index_map'] = ''
_required_data_df

,answers.allergies,answers.bp1Diastole,answers.bp1Systole,answers.rhesusType,answers.weeklyExercise,answers.smokingHabit,profileId,_index_map,answers.familyDiagnosis,answers.civilStatus,gender,dateCreated,createdBy,_id,id
0,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,,"['Kidney disease', 'Kidney disease', 'Kidney d...",Single,Male,07/22/2018 at 08:37:43 PM GMT+08:00,kristel@gmail.com,758c968a-4bad-4ffd-83ed-16b105d12404,758c968a-4bad-4ffd-83ed-16b105d12404
1,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,,,Single,,,,329aef98-ebb4-4188-8a45-08bd954d714d,329aef98-ebb4-4188-8a45-08bd954d714d
2,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,,,,,,,ab191553-b31f-41a7-9436-dd1298026f24,ab191553-b31f-41a7-9436-dd1298026f24
3,,,,Wala ko kabalo,,empty,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,,Single,Babae,07/16/2018 at 11:53:05 AM GMT+08:00,geralyn@gmail.com,cb115c52-5303-4f16-a679-dd8ff5df31eb,cb115c52-5303-4f16-a679-dd8ff5df31eb
4,,,,Wala ko kabalo,,empty,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,,Single,,,,37fbcb0a-0fab-4c7e-9ac6-9c371baf096d,37fbcb0a-0fab-4c7e-9ac6-9c371baf096d
5,,,,Wala ko kabalo,,Hindi,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,,Single,,,,a3757c6e-e13e-4ddd-b7ab-ad2ff1b40a5e,a3757c6e-e13e-4ddd-b7ab-ad2ff1b40a5e
6,,,,empty,0 to 1 hour,Never,0006de3c-57ea-46d2-89b8-e606533d3d15,,"['ubo/sipon', 'Stroke']",Single,Male,07/25/2018 at 10:35:28 AM GMT+08:00,kristel@gmail.com,e26302bb-49d4-4c94-a291-fa28073b9675,e26302bb-49d4-4c94-a291-fa28073b9675
7,,,,empty,0 to 1 hour,Never,0006de3c-57ea-46d2-89b8-e606533d3d15,,,Single,,,,c2bf04be-e637-4b72-ab5d-696ed8ada1db,c2bf04be-e637-4b72-ab5d-696ed8ada1db
8,,80,110,empty,0 to 1 hour,Stopped for more than a year,000c23e6-6f0b-45ab-a0b5-e5d19dc023ad,,"['Diabetes', 'Respiratory disease']",Married,Male,07/25/2018 at 11:06:17 AM GMT+08:00,silver@gmail.com,2a1c6260-a84c-441a-8d02-e3e5d74fd48b,2a1c6260-a84c-441a-8d02-e3e5d74fd48b
9,no,80,110,Wala ko kabalo,0 to 1 Oras,Hindi,000cb28d-04d8-4665-90e6-fb1e2fe3c11c,,,Widowed,Babae,07/14/2018 at 11:45:05 AM GMT+08:00,bonbon@gmail.com,6ce8af76-f6f0-4021-9662-6e75ce696d00,6ce8af76-f6f0-4021-9662-6e75ce696d00


## GET THE NEW FIELDS WITH THE DEFAULT VALUES

In [215]:
new_fields_df = _mapping_df[_mapping_df['data_type'] == 'new'][['destination_key', 'default_value']]
new_fields_df

,destination_key,default_value
12,answers.consentBloodTest,N
13,answers.dailyFiberIntake,
14,dateUpdated,2019-06-27T21:00:51.934+08:00
15,formId,aF5AuuQSBToFGyuFz9HGi9
16,formName,2.2 - AQM HealthInfo Questions V1
17,type,profile-related-form


## APPEND THE NEW FIELDS AS COLUMN

In [216]:
for index,row in new_fields_df.iterrows():
    _header = row['destination_key']
    _value = row['default_value']
    
    _required_data_df[_header] = _value

_required_data_df = _required_data_df.reset_index(drop=True)
_required_data_df.head(5)

,answers.allergies,answers.bp1Diastole,answers.bp1Systole,answers.rhesusType,answers.weeklyExercise,answers.smokingHabit,profileId,_index_map,answers.familyDiagnosis,answers.civilStatus,...,dateCreated,createdBy,_id,id,answers.consentBloodTest,answers.dailyFiberIntake,dateUpdated,formId,formName,type
0,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,,"['Kidney disease', 'Kidney disease', 'Kidney d...",Single,...,07/22/2018 at 08:37:43 PM GMT+08:00,kristel@gmail.com,758c968a-4bad-4ffd-83ed-16b105d12404,758c968a-4bad-4ffd-83ed-16b105d12404,N,,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,profile-related-form
1,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,,,Single,...,,,329aef98-ebb4-4188-8a45-08bd954d714d,329aef98-ebb4-4188-8a45-08bd954d714d,N,,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,profile-related-form
2,,,,empty,0 to 1 hour,Never,0003ff38-28fb-4005-9437-d276cbb9da4d,,,,...,,,ab191553-b31f-41a7-9436-dd1298026f24,ab191553-b31f-41a7-9436-dd1298026f24,N,,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,profile-related-form
3,,,,Wala ko kabalo,,empty,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,,Single,...,07/16/2018 at 11:53:05 AM GMT+08:00,geralyn@gmail.com,cb115c52-5303-4f16-a679-dd8ff5df31eb,cb115c52-5303-4f16-a679-dd8ff5df31eb,N,,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,profile-related-form
4,,,,Wala ko kabalo,,empty,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,,,Single,...,,,37fbcb0a-0fab-4c7e-9ac6-9c371baf096d,37fbcb0a-0fab-4c7e-9ac6-9c371baf096d,N,,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,profile-related-form


## SORT COLUMN HEADERS

In [217]:
_sorted_columns = sorted(list(_required_data_df.columns))
_required_data_df = _required_data_df[_sorted_columns]
_required_data_df

,_id,_index_map,answers.allergies,answers.bp1Diastole,answers.bp1Systole,answers.civilStatus,answers.consentBloodTest,answers.dailyFiberIntake,answers.familyDiagnosis,answers.rhesusType,...,answers.weeklyExercise,createdBy,dateCreated,dateUpdated,formId,formName,gender,id,profileId,type
0,758c968a-4bad-4ffd-83ed-16b105d12404,,,,,Single,N,,"['Kidney disease', 'Kidney disease', 'Kidney d...",empty,...,0 to 1 hour,kristel@gmail.com,07/22/2018 at 08:37:43 PM GMT+08:00,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,Male,758c968a-4bad-4ffd-83ed-16b105d12404,0003ff38-28fb-4005-9437-d276cbb9da4d,profile-related-form
1,329aef98-ebb4-4188-8a45-08bd954d714d,,,,,Single,N,,,empty,...,0 to 1 hour,,,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,,329aef98-ebb4-4188-8a45-08bd954d714d,0003ff38-28fb-4005-9437-d276cbb9da4d,profile-related-form
2,ab191553-b31f-41a7-9436-dd1298026f24,,,,,,N,,,empty,...,0 to 1 hour,,,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,,ab191553-b31f-41a7-9436-dd1298026f24,0003ff38-28fb-4005-9437-d276cbb9da4d,profile-related-form
3,cb115c52-5303-4f16-a679-dd8ff5df31eb,,,,,Single,N,,,Wala ko kabalo,...,,geralyn@gmail.com,07/16/2018 at 11:53:05 AM GMT+08:00,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,Babae,cb115c52-5303-4f16-a679-dd8ff5df31eb,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,profile-related-form
4,37fbcb0a-0fab-4c7e-9ac6-9c371baf096d,,,,,Single,N,,,Wala ko kabalo,...,,,,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,,37fbcb0a-0fab-4c7e-9ac6-9c371baf096d,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,profile-related-form
5,a3757c6e-e13e-4ddd-b7ab-ad2ff1b40a5e,,,,,Single,N,,,Wala ko kabalo,...,,,,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,,a3757c6e-e13e-4ddd-b7ab-ad2ff1b40a5e,0004056c-ec55-4c4d-ba14-9ccbd0ddfa81,profile-related-form
6,e26302bb-49d4-4c94-a291-fa28073b9675,,,,,Single,N,,"['ubo/sipon', 'Stroke']",empty,...,0 to 1 hour,kristel@gmail.com,07/25/2018 at 10:35:28 AM GMT+08:00,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,Male,e26302bb-49d4-4c94-a291-fa28073b9675,0006de3c-57ea-46d2-89b8-e606533d3d15,profile-related-form
7,c2bf04be-e637-4b72-ab5d-696ed8ada1db,,,,,Single,N,,,empty,...,0 to 1 hour,,,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,,c2bf04be-e637-4b72-ab5d-696ed8ada1db,0006de3c-57ea-46d2-89b8-e606533d3d15,profile-related-form
8,2a1c6260-a84c-441a-8d02-e3e5d74fd48b,,,80,110,Married,N,,"['Diabetes', 'Respiratory disease']",empty,...,0 to 1 hour,silver@gmail.com,07/25/2018 at 11:06:17 AM GMT+08:00,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,Male,2a1c6260-a84c-441a-8d02-e3e5d74fd48b,000c23e6-6f0b-45ab-a0b5-e5d19dc023ad,profile-related-form
9,6ce8af76-f6f0-4021-9662-6e75ce696d00,,no,80,110,Widowed,N,,,Wala ko kabalo,...,0 to 1 Oras,bonbon@gmail.com,07/14/2018 at 11:45:05 AM GMT+08:00,2019-06-27T21:00:51.934+08:00,aF5AuuQSBToFGyuFz9HGi9,2.2 - AQM HealthInfo Questions V1,Babae,6ce8af76-f6f0-4021-9662-6e75ce696d00,000cb28d-04d8-4665-90e6-fb1e2fe3c11c,profile-related-form


# WRITE TO CSV FINALIZED


In [218]:
_required_data_df.to_csv(merged_dir + _output_filename + '.csv', encoding='utf-8', mode='w', header=True,index=False)